In [1]:
import numpy as np
import pandas as pd
import re
import string
import unittest
from collections import Counter
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
np.random.seed(2018)

import nltk

from gensim import corpora

Lda = gensim.models.ldamodel.LdaModel

num_topic=5 # number of topics selected.
k=5  #k value for Knn

class Tokenizer:
    def __init__(self, text):
        self._text = text
        self._tokenized_text = []

    def tokenize(self):
        # Remove punctuation, empty elements, numbers, and dates
        pattern = re.compile('[0-9]+')
        self._tokenized_text = [''.join(c for c in s if c not in string.punctuation) for s in self._text]
        self._tokenized_text[:] = [word for word in self._tokenized_text if not pattern.match(word) and word != '']

        return self._tokenized_text


class RemoveStopWords:
    def __init__(self, text):
        self._text = text
        self._stopwords = []

    def removestopwords(self):
        with open('C:/Users/baoji/Documents/src/stopwords.txt', 'r') as g:
            self._stopwords = g.read().splitlines()
        for word in self._stopwords:
            self._text = [value for value in self._text if value.lower() != word]

        return self._text
class dataset:
    def __init__(self, txt1,txt2):
        self._txt1 = txt1
        self._txt2 = txt2
        self._train=pd.DataFrame()
        self._validate=pd.DataFrame()
        self._test=pd.DataFrame()
        


    def bdsproject_merge(self):
        #merge dataset by userID
        data = pd.read_csv(self._txt1, sep="\t", header=None)
        data.columns = ["userID","b", "rating", "lable", "date"]
        data=data.drop(['b', "rating", 'date'],axis=1)
        data2 = pd.read_csv(self._txt2, sep="\t", header=None)
        data2.columns = ["userID","b", "date", "content"]
        data2=data2.drop(['b', 'date'],axis=1)
        result=data.set_index('userID').join(data2.set_index('userID'))
        
        #rename column
        result.columns = [  "label", "content"]
        
        #make the dataset balanced
        fakedata1=result.loc[result["label"]==-1]
        fakedata2=fakedata1.sample(n=10000)
        nfakedata1=result.loc[result["label"]==1]
        nfakedata2=nfakedata1.sample(n=10000)
        nfakedata3=nfakedata1.sample(n=105000)
        result2=pd.concat([fakedata1,nfakedata3],ignore_index=True)
        
        #split dataset to train, validate and test and save it as csv.
    
        train, validate, test = np.split(result2.sample(frac=1), [int(.6*len(result2)), int(.8*len(result2))])
        test=test.sample(n=10000)
        final=pd.concat([fakedata2,nfakedata2,test])
        print(final)
        

        return final
def textprocess(data):
    data=data.content.values.tolist()
    lemmatizer = WordNetLemmatizer()
    wordCollect=[]
    for word in data:
        word=str(word)
        wordCollect.append(word.split())
    collect=[]
    for data in wordCollect:
        preprocessedlist=[]
        temptext = Tokenizer(data)
        cleantext = temptext.tokenize()
        temptext = RemoveStopWords(cleantext)
        cleantext = temptext.removestopwords()
        lemma_text = []

        for word in list(cleantext):
            new_word = lemmatizer.lemmatize(word)
            lemma_text.append(new_word)

        for word in lemma_text:
            preprocessedlist.append(word)
        collect.append(preprocessedlist)
    return collect


def lda(collect):
    # dic for terms
    dictionary = corpora.Dictionary(collect)

    # DT matrix
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in collect]
    return dictionary, doc_term_matrix
Prep = dataset('C:/Users/baoji/Documents/src/metadata.txt','C:/Users/baoji/Documents/src/reviewContent.txt')

f=Prep.bdsproject_merge()
fc=textprocess(f)

mark=f.label.values.tolist()
dictionary_f,corpus_f=lda(fc)
lda_f = Lda(corpus_f, num_topics=num_topic, id2word = dictionary_f, passes=50)
topics = lda_f.show_topics()
for topic in topics:
    print (topic)
    
all_topics = lda_f.get_document_topics(corpus_f, per_word_topics=True)
i=0
d_t=[]
for doc_topics, word_topics, phi_values in all_topics:
    i+=1
    d_t.append(doc_topics)
    print ('Review '+str(i)+' topics:', doc_topics)
    
print(lda_f)

vec=[]
for term in d_t:
    temp=[]
    for i in range(0,num_topic):
        temp.append(0)
    for topic in term:
        temp[int(topic[0])]=topic[1]
    vec.append(temp)
    
def Knn(a, train, k):
    i=0
    mi=[]
    index=[]
    for x in range(0,k):
        mi.append(100)
        index.append(0)
    for l in train:
        dis=dist(a,l)
        if dis<mi[-1]:
            mi[k-1]=dis
            index[k-1]=i
            j=k-1
            while mi[j]<mi[j-1] and j>=1:
                    temp=mi[j]
                    mi[j]=mi[j-1]
                    mi[j-1]=temp
                    temp=index[j]
                    index[j]=index[j-1]
                    index[j-1]=temp
                    j-=1
            
        i+=1
    return index

def dist(a,b):
    dis=0
    for i in range(0,num_topic):
        dis+=((a[i]-b[i])**2)**0.5
    return dis




        label                                            content
923        -1  This restaurant was quite a pleasant surprise....
4168       -1  High end sushi. The best in New York. I am a r...
60625      -1  I'm speechless everything was great from the s...
6541       -1  This place is a very cramped hole in the whole...
15357      -1  OMG...I must be either SUPER picky or the food...
11521      -1  All I can say is fideos (short pieces of pasta...
71459      -1  Eat here!  The food, atmosphere and service ar...
4137       -1  We had the address of this restaurant and were...
17800      -1  Better than Chipotle in terms of freshness and...
7052       -1  An adventure to the WV. I found Cafe Condesa o...
36449      -1  Over hyped, long lines for an average burger. ...
37784      -1  After reading the reviews for Grammercy Tavern...
8980       -1  Last time i ordered the spinach gnocchi in the...
3363       -1  This place is so overrated. I don't understand...
8774       -1  Fish tacos

Review 75 topics: [(3, 0.9032146), (4, 0.091078505)]
Review 76 topics: [(0, 0.015605334), (1, 0.643142), (2, 0.015545773), (3, 0.31029055), (4, 0.015416396)]
Review 77 topics: [(0, 0.01679049), (1, 0.8308849), (2, 0.0168116), (3, 0.01696395), (4, 0.118549064)]
Review 78 topics: [(0, 0.025176376), (1, 0.89907), (2, 0.025162214), (3, 0.025401162), (4, 0.025190245)]
Review 79 topics: [(1, 0.6347632), (3, 0.080731615), (4, 0.26787528)]
Review 80 topics: [(0, 0.026952013), (1, 0.68340373), (3, 0.28179422)]
Review 81 topics: [(0, 0.06677086), (1, 0.7312768), (2, 0.06860813), (3, 0.06667208), (4, 0.066672124)]
Review 82 topics: [(1, 0.5131679), (2, 0.46234939)]
Review 83 topics: [(1, 0.24511915), (2, 0.53993106), (3, 0.12762536), (4, 0.080075555)]
Review 84 topics: [(0, 0.17511298), (1, 0.797119)]
Review 85 topics: [(0, 0.013292174), (1, 0.12537272), (2, 0.0132880695), (3, 0.83476186), (4, 0.013285175)]
Review 86 topics: [(0, 0.5131338), (1, 0.018744638), (2, 0.14836463), (3, 0.301446), (4, 0

Review 261 topics: [(1, 0.4452573), (3, 0.38831), (4, 0.15877186)]
Review 262 topics: [(1, 0.5725751), (2, 0.3224144), (3, 0.0942874)]
Review 263 topics: [(0, 0.2024414), (1, 0.37397292), (2, 0.2499555), (3, 0.16748425)]
Review 264 topics: [(0, 0.7980718), (1, 0.050790716), (2, 0.050505947), (3, 0.050321348), (4, 0.050310165)]
Review 265 topics: [(0, 0.38629842), (1, 0.050323095), (2, 0.4633283), (3, 0.050024997), (4, 0.05002516)]
Review 266 topics: [(1, 0.22468518), (3, 0.74985594)]
Review 267 topics: [(0, 0.01875482), (1, 0.8144621), (2, 0.018815657), (3, 0.018675324), (4, 0.12929212)]
Review 268 topics: [(1, 0.7988629), (2, 0.18620193)]
Review 269 topics: [(0, 0.7820293), (1, 0.02550723), (2, 0.026579682), (3, 0.026658135), (4, 0.13922566)]
Review 270 topics: [(0, 0.015573206), (1, 0.93732405), (2, 0.015713278), (3, 0.015847191), (4, 0.015542281)]
Review 271 topics: [(1, 0.97431314)]
Review 272 topics: [(0, 0.014262953), (1, 0.670502), (2, 0.014198348), (3, 0.28687936), (4, 0.014157

Review 453 topics: [(0, 0.05455044), (1, 0.043381285), (2, 0.23535597), (3, 0.6646284)]
Review 454 topics: [(0, 0.35425475), (1, 0.26781592), (2, 0.020310136), (3, 0.020404292), (4, 0.33721495)]
Review 455 topics: [(1, 0.8181623), (2, 0.10775997), (4, 0.058441017)]
Review 456 topics: [(0, 0.042745896), (1, 0.57847375), (2, 0.14774263), (3, 0.19300516), (4, 0.03803257)]
Review 457 topics: [(0, 0.014658303), (1, 0.9416885), (2, 0.014587015), (3, 0.014575057), (4, 0.0144911185)]
Review 458 topics: [(1, 0.4467433), (2, 0.5296935)]
Review 459 topics: [(0, 0.23265922), (1, 0.13905309), (2, 0.40904877), (3, 0.020485284), (4, 0.19875367)]
Review 460 topics: [(0, 0.27492663), (1, 0.018768137), (2, 0.018855745), (3, 0.66848797), (4, 0.018961523)]
Review 461 topics: [(0, 0.06143627), (1, 0.27761072), (2, 0.31114602), (3, 0.031490725), (4, 0.31831622)]
Review 462 topics: [(0, 0.19040012), (1, 0.7281275), (2, 0.0349115), (3, 0.04331741)]
Review 463 topics: [(0, 0.1004419), (3, 0.8853718)]
Review 46

Review 550 topics: [(1, 0.1900224), (3, 0.54542786), (4, 0.25253654)]
Review 551 topics: [(3, 0.97628343)]
Review 552 topics: [(0, 0.05084244), (1, 0.32377636), (2, 0.050091416), (3, 0.52204376), (4, 0.05324601)]
Review 553 topics: [(0, 0.102336794), (1, 0.61163485), (2, 0.2259596), (4, 0.054311205)]
Review 554 topics: [(0, 0.030196978), (1, 0.030741174), (2, 0.030402297), (3, 0.22870703), (4, 0.6799525)]
Review 555 topics: [(0, 0.6196233), (1, 0.014724699), (2, 0.014535181), (3, 0.3365612), (4, 0.014555598)]
Review 556 topics: [(1, 0.7449769), (2, 0.23455217)]
Review 557 topics: [(1, 0.7530964), (2, 0.06947974), (4, 0.16036049)]
Review 558 topics: [(1, 0.60399103), (2, 0.3742808)]
Review 559 topics: [(0, 0.011318698), (1, 0.50208986), (2, 0.46393657), (3, 0.011347585), (4, 0.011307283)]
Review 560 topics: [(0, 0.046774168), (1, 0.64739984), (3, 0.2975856)]
Review 561 topics: [(0, 0.24167357), (1, 0.53198135), (2, 0.12166006), (3, 0.09276337), (4, 0.0119217085)]
Review 562 topics: [(0,

...

Review 29304 topics: [(0, 0.3197839), (1, 0.26770255), (3, 0.28402367), (4, 0.12529227)]
Review 29305 topics: [(0, 0.4442673), (1, 0.21352556), (2, 0.012862816), (3, 0.31671065), (4, 0.012633646)]
Review 29306 topics: [(0, 0.12611908), (1, 0.771571), (4, 0.0963779)]
Review 29307 topics: [(0, 0.025406154), (1, 0.35104144), (2, 0.025169574), (3, 0.5731954), (4, 0.025187457)]
Review 29308 topics: [(0, 0.37155616), (1, 0.32406953), (2, 0.28506798)]
Review 29309 topics: [(0, 0.06705469), (1, 0.087905705), (2, 0.3286487), (3, 0.3692578), (4, 0.14713314)]
Review 29310 topics: [(2, 0.3628445), (3, 0.43055636), (4, 0.19478548)]
Review 29311 topics: [(1, 0.5617348), (3, 0.27891266), (4, 0.14368667)]
Review 29312 topics: [(1, 0.1814707), (2, 0.15259972), (3, 0.5838645), (4, 0.08062798)]
Review 29313 topics: [(0, 0.4015927), (1, 0.12664616), (2, 0.2459033), (3, 0.20595922), (4, 0.01989862)]
Review 29314 topics: [(1, 0.9135229), (3, 0.06948717)]
Review 29315 topics: [(0, 0.0250547), (1, 0.02507993)

Review 29471 topics: [(0, 0.27562606), (1, 0.68646705), (2, 0.012702914), (3, 0.0125659015), (4, 0.012638115)]
Review 29472 topics: [(1, 0.55975676), (2, 0.4216503)]
Review 29473 topics: [(1, 0.28543007), (2, 0.3231085), (3, 0.31103423), (4, 0.07045041)]
Review 29474 topics: [(0, 0.490853), (1, 0.09952064), (2, 0.16755725), (3, 0.23924252)]
Review 29475 topics: [(0, 0.23096977), (1, 0.16614525), (2, 0.5717927), (3, 0.01558989), (4, 0.015502371)]
Review 29476 topics: [(1, 0.225256), (2, 0.6624032), (3, 0.10024873)]
Review 29477 topics: [(1, 0.3670763), (3, 0.5640783), (4, 0.053196754)]
Review 29478 topics: [(0, 0.028931621), (1, 0.028956568), (2, 0.028890433), (3, 0.8844771), (4, 0.028744334)]
Review 29479 topics: [(0, 0.08874221), (1, 0.74964345), (3, 0.1502741)]
Review 29480 topics: [(0, 0.7315002), (2, 0.051370177), (3, 0.16989094), (4, 0.04216408)]
Review 29481 topics: [(0, 0.24799772), (1, 0.025234943), (2, 0.025353884), (3, 0.025152292), (4, 0.6762612)]
Review 29482 topics: [(1, 0

Review 29635 topics: [(0, 0.12198797), (1, 0.33065206), (2, 0.2934804), (3, 0.24202873), (4, 0.011850846)]
Review 29636 topics: [(0, 0.050759427), (1, 0.22755197), (2, 0.04372135), (3, 0.555239), (4, 0.12272818)]
Review 29637 topics: [(0, 0.022205511), (1, 0.2893815), (2, 0.508425), (3, 0.12618577), (4, 0.053802207)]
Review 29638 topics: [(0, 0.014554778), (1, 0.56808597), (2, 0.014695118), (3, 0.388308), (4, 0.014356172)]
Review 29639 topics: [(0, 0.2693012), (1, 0.34626374), (3, 0.29953164), (4, 0.075583495)]
Review 29640 topics: [(0, 0.17454313), (1, 0.20334317), (2, 0.27090514), (3, 0.23907909), (4, 0.11212949)]
Review 29641 topics: [(0, 0.020203806), (1, 0.020619143), (2, 0.020197354), (3, 0.91870356), (4, 0.02027613)]
Review 29642 topics: [(1, 0.32660598), (2, 0.43937153), (3, 0.202187), (4, 0.029867258)]
Review 29643 topics: [(1, 0.37502518), (2, 0.0734887), (3, 0.54425454)]
Review 29644 topics: [(0, 0.040130876), (1, 0.62720585), (2, 0.0400183), (3, 0.040220577), (4, 0.25242442

Review 29806 topics: [(0, 0.48746064), (2, 0.065023385), (3, 0.4409843)]
Review 29807 topics: [(0, 0.0301081), (1, 0.028851677), (2, 0.029741652), (3, 0.8822999), (4, 0.028998673)]
Review 29808 topics: [(0, 0.1561186), (1, 0.58828455), (3, 0.2402833)]
Review 29809 topics: [(0, 0.016829375), (1, 0.93265545), (2, 0.016866362), (3, 0.016857779), (4, 0.01679105)]
Review 29810 topics: [(0, 0.08860433), (1, 0.35532737), (2, 0.5419028)]
Review 29811 topics: [(0, 0.39817533), (1, 0.019043336), (2, 0.24104118), (3, 0.019805403), (4, 0.32193476)]
Review 29812 topics: [(0, 0.33557636), (1, 0.0112638585), (2, 0.011255088), (3, 0.6305502), (4, 0.01135451)]
Review 29813 topics: [(0, 0.598089), (1, 0.3729077)]
Review 29814 topics: [(0, 0.36206248), (1, 0.07727521), (3, 0.53601706), (4, 0.021594174)]
Review 29815 topics: [(0, 0.012695189), (1, 0.94934344), (2, 0.012682863), (3, 0.012647849), (4, 0.012630655)]
Review 29816 topics: [(1, 0.4443238), (2, 0.48766202), (4, 0.051870506)]
Review 29817 topics:

Review 29968 topics: [(0, 0.16989154), (1, 0.42604178), (3, 0.3847203)]
Review 29969 topics: [(0, 0.074689396), (1, 0.6536931), (2, 0.20891677), (3, 0.05349626)]
Review 29970 topics: [(0, 0.07887009), (1, 0.23271884), (2, 0.25104457), (3, 0.4068823), (4, 0.030484172)]
Review 29971 topics: [(0, 0.5757347), (1, 0.3033199), (2, 0.040241174), (3, 0.04016705), (4, 0.04053724)]
Review 29972 topics: [(0, 0.010093819), (1, 0.87449294), (2, 0.09502568), (3, 0.010215071), (4, 0.010172527)]
Review 29973 topics: [(0, 0.4290509), (1, 0.10569189), (3, 0.4506831)]
Review 29974 topics: [(0, 0.7077961), (1, 0.24133451), (2, 0.016845407), (3, 0.017270282), (4, 0.016753675)]
Review 29975 topics: [(0, 0.6361519), (2, 0.13393989), (3, 0.21792139)]
Review 29976 topics: [(0, 0.30847737), (1, 0.20695144), (2, 0.1591962), (3, 0.14211808), (4, 0.18325691)]
Review 29977 topics: [(0, 0.01846837), (1, 0.018847516), (2, 0.018600006), (3, 0.92581016), (4, 0.018273996)]
Review 29978 topics: [(0, 0.033374827), (1, 0.3

In [2]:
train=vec[:20000]
test=vec[20000:]
i=20000
tf=0
tt=0
ft=0
ff=0
for term in test:
    best=Knn(term,train,k)
    for index in best:
        count_t=0
        count_f=0
        if mark[index]==-1:
            count_f+=1
        else:
            count_t+=1
    if count_f>count_t:
        r=-1
    else:
        r=1
    if r==mark[i]:
        print(True)
        if r==1:
            tt+=1
        else:
            ff+=1
    else:
        print(False, best[0], best[1], best[2], r)
        if r==1:
            tf+=1
        else:
            ft+=1
    i+=1

False 10024 14873 13946 1
True
True
True
True
True
True
False 18068 6696 4402 -1
False 1488 10185 14243 -1
False 15444 3597 4836 -1
False 3213 17548 3019 -1
True
True
True
True
True
False 1934 1954 1718 1
True
False 970 6455 2387 1
False 11135 11471 14055 -1
False 1426 19679 6781 -1
False 5630 6214 9087 1
True
True
True
True
False 12094 3412 8810 -1
False 9305 7042 5135 1
True
True
True
True
True
False 9736 16388 11055 1
False 14080 15381 17609 -1
False 5779 6884 2693 -1
True
True
True
True
True
True
True
True
False 10579 10255 15913 -1
False 19500 13411 17655 -1
False 6973 890 3280 1
True
False 7084 7367 562 -1
False 12360 3662 16779 -1
False 12315 16523 12254 -1
False 12751 12479 18013 -1
True
True
True
False 6539 1545 643 -1
True
True
False 12075 5166 6644 -1
True
False 16955 10371 15749 1
True
True
False 11947 4573 4688 -1
True
False 2496 5104 9430 -1
True
False 11476 312 12349 1
True
False 16833 10829 13901 -1
True
False 3908 4234 7272 1
True
True
False 14933 14782 7374 -1
True
Tr

True
True
True
True
False 14526 15239 1585 1
True
True
True
False 3870 5928 7404 1
True
True
True
True
True
True
False 5931 19934 3659 -1
False 7630 8495 12186 1
True
False 456 7141 5746 1
True
False 1357 5650 807 -1
True
True
True
False 10553 17756 5739 -1
True
False 3909 16987 5458 1
True
True
True
True
False 15559 9877 18506 -1
False 1949 10608 19314 1
False 4498 9099 919 1
False 11702 15652 18339 1
True
False 400 4666 1745 1
True
True
False 11639 8359 10808 -1
False 9175 2516 11921 -1
False 10025 4850 11442 1
False 7174 14783 10011 -1
False 17922 11904 17436 -1
False 5102 6591 5868 -1
False 1408 15312 3948 -1
True
False 571 10009 6736 -1
False 5989 13499 17314 -1
True
False 11928 6757 223 -1
False 4447 8950 1354 -1
False 16735 12932 19021 -1
False 528 5375 16359 -1
True
True
True
True
True
False 133 17990 4880 -1
True
False 13901 16833 10829 -1
True
False 5411 16185 14835 -1
True
False 6737 17706 12540 -1
False 7579 9859 11440 1
True
False 4717 4589 2092 -1
False 9264 9412 11767 1


True
False 15305 19064 17616 -1
True
True
False 5328 17697 3413 -1
True
True
False 87 1663 4805 -1
True
False 15133 13019 11468 -1
True
False 15327 10564 4489 1
True
True
False 17191 18295 11671 1
True
False 11408 8354 9960 -1
False 5158 15184 14483 1
True
False 10117 17912 1753 -1
False 2485 3990 11721 1
False 9923 9330 7490 -1
False 16889 15966 2547 1
True
False 19623 778 15353 -1
False 3980 7591 9165 1
True
False 15808 15236 12200 1
False 5716 994 6142 -1
False 7658 4891 8049 -1
True
True
False 11787 5800 9716 -1
False 1731 15145 5367 -1
True
True
False 7478 8661 698 1
False 14489 17780 1128 -1
True
True
True
False 7221 6372 8951 -1
False 11504 9724 19831 1
False 12061 17112 17044 1
False 11542 13529 5504 -1
True
True
True
False 738 7819 9167 -1
False 9813 19580 16830 1
True
True
False 12920 17269 19891 1
True
True
True
True
True
True
False 14857 15629 18472 -1
False 10083 3207 8558 1
False 8494 17196 12137 1
True
False 2666 4022 7950 -1
True
False 2653 850 1326 1
True
False 2856 77

True
False 3448 17420 2584 -1
False 6232 18272 10669 1
True
True
False 15087 17716 12775 -1
True
True
False 11979 15928 10763 -1
True
True
False 10069 10731 11193 1
True
True
True
True
True
False 2454 14562 11578 -1
False 2252 440 14402 1
True
False 19435 9614 10113 1
True
False 7199 4322 14455 1
True
False 2626 4671 3740 1
True
True
False 17423 19501 15796 -1
True
True
True
True
True
True
True
False 11664 4918 4099 1
True
True
True
False 1750 50 8988 -1
False 1515 12667 19351 1
True
True
True
True
False 8460 18035 11331 1
False 10390 10630 15136 -1
True
False 9089 3975 3711 -1
True
False 13285 6144 16487 -1
False 13015 19055 18119 1
False 7982 2452 1669 1
False 6392 4421 8896 1
True
True
False 1883 1195 16051 1
True
True
False 4629 8057 3738 1
True
True
True
False 8039 14703 10777 1
True
True
False 5950 16631 1885 1
True
True
True
True
True
True
False 12373 5670 17559 -1
False 8635 92 12310 -1
True
True
True
True
True
False 13124 14611 11153 1
True
False 3822 15256 6298 -1
True
False 

True
True
False 10763 11810 11677 1
True
True
True
False 6277 16043 3585 -1
True
True
False 3487 6474 15910 1
False 1294 9522 1269 -1
False 9526 2932 2517 -1
True
True
True
False 6821 16751 13942 1
True
False 7147 4113 3778 1
False 7778 18639 12084 -1
True
False 955 6074 3252 1
True
True
True
True
True
False 13450 15585 17839 1
False 2751 1792 6364 1
False 9594 8801 3907 1
False 12288 11003 17209 1
True
False 2079 1182 6547 1
True
True
False 9830 11309 10402 1
False 11237 2729 5770 -1
True
True
True
True
True
False 18220 15983 4199 -1
True
False 10327 18923 9104 -1
False 13419 9991 55 1
False 5516 19769 8712 -1
False 16826 18209 13215 -1
True
False 5676 23 13601 -1
True
True
True
True
True
True
True
True
False 19020 19175 17432 1
True
True
False 17672 11858 10758 -1
True
True
True
True
True
False 18822 13781 1573 -1
True
True
True
True
False 3591 12392 9663 -1
False 9343 5142 8177 -1
False 5548 1816 8489 -1
False 6777 162 9603 1
True
True
False 11356 13914 18863 -1
False 19098 2780 773

...

False 17517 11954 14497 -1
True
False 8103 15984 13246 1
True
True
True
True
True
False 9546 5674 6976 -1
True
False 16758 14516 17784 -1
True
False 12084 7778 16508 -1
True
True
True
True
True
True
False 1782 5324 7145 1
False 6665 8883 5077 1
False 204 4415 16076 1
True
True
False 201 707 5093 -1
False 11710 3319 4738 -1
False 14295 17116 17655 -1
True
True
True
True
True
True
False 185 13011 5910 1
True
True
False 9214 8833 12615 1
True
False 19528 16137 5881 -1
True
True
True
True
False 10952 14258 12178 1
False 4175 5560 2848 1
False 15136 10390 10429 -1
True
False 11721 14054 2882 -1
False 38 2167 11054 1
True
False 320 10750 1263 -1
True
True
True
False 8450 1407 18645 -1
True
True
True
True
False 2584 17420 10984 1
False 7425 2381 12164 -1
True
False 9198 16380 4507 -1
False 6653 8846 8575 1
True
False 10622 17461 17650 -1
False 9404 7702 7485 1
False 8409 17137 14791 -1
False 18024 15189 6079 -1
True
True
True
False 10916 10088 13770 -1
True
True
True
False 8630 4613 3234 -1
T

False 15788 14681 9768 -1
False 6533 4678 17870 -1
True
True
True
False 1232 5227 757 1
True
False 19385 10512 10811 -1
False 1186 18492 15141 -1
False 14104 7324 19824 -1
True
True
True
True
True
True
False 14684 4044 19994 1
False 3083 9868 18018 -1
True
False 15000 4672 4379 -1
False 14334 17836 16988 1
False 11463 17726 17601 1
False 15239 14526 16263 1
True
False 11069 12051 10552 1
True
True
True
False 3804 5862 7893 -1
False 4705 9889 3910 -1
True
False 15164 11728 7803 1
True
True
True
False 18144 11222 11012 -1
True
False 6665 8883 5077 1
False 15732 18363 14996 -1
True
False 18319 3104 12362 -1
True
False 16332 7636 4424 -1
True
True
False 3068 14042 17829 -1
False 7743 2858 3217 1
True
True
False 6768 6362 3115 1
True
False 15133 1245 15955 -1
True
False 1588 17527 19352 1
True
False 6329 1803 13575 -1
True
False 8666 6523 2133 1
True
False 6858 3652 18573 1
False 14027 589 2747 -1
True
False 15643 10016 1918 -1
True
True
False 13821 1427 7272 -1
True
True
True
True
True
Tru

True
False 13749 14967 3080 -1
True
False 12220 16897 19349 -1
True
True
True
False 4969 18122 18862 -1
True
True
False 5203 1424 1160 -1
True
True
True
False 7429 14893 19029 1
False 14789 17797 13296 1
False 8809 14195 18340 -1
True
True
True
True
True
False 11693 11240 19567 -1
False 4490 15558 14245 -1
False 16260 8089 13855 -1
False 11498 7581 1277 1
True
True
False 8055 12871 6514 -1
False 13945 15559 6499 -1
True
True
True
True
True
False 8247 5848 12414 1
True
False 13901 16833 10829 -1
True
False 17821 3391 1243 1
False 14925 11923 17629 1
True
True
True
True
True
False 5584 19576 5057 1
False 1002 2516 3329 1
True
False 7997 1009 6417 1
True
False 2823 18412 17456 1
True
True
True
True
False 13576 19336 2040 -1
True
True
True
True
True
False 13803 607 11606 -1
True
True
False 9853 16439 11360 1
False 18197 14799 6453 -1
True
True
True
True
False 17745 13234 14998 1
False 12144 9367 146 -1
True
True
True
True
False 980 8092 6615 1
False 13953 13853 9924 1
False 6321 2672 15561

True
True
True
True
True
True
True
False 7068 7526 8814 1
True
True
False 14508 16232 16798 -1
False 16643 11771 4095 1
True
False 411 2604 17216 1
False 3727 10021 5000 -1
False 12028 16973 13058 -1
True
True
True
True
True
False 11936 11759 19731 -1
True
True
True
True
False 2567 15560 3093 1
True
True
False 8059 19241 17180 1
False 18459 1 9398 -1
True
True
True
True
True
True
False 9498 19007 5253 1
True
False 12456 5262 4844 1
True
True
True
False 6192 4237 3306 -1
True
True
True
False 8578 9613 513 -1
True
False 2466 8374 7674 1
False 19788 8059 19241 -1
False 15 9731 6203 1
True
False 14239 17503 10941 -1
True
True
True
True
True
True
False 2998 9464 5026 1
True
True
True
True
True
True
False 13255 3266 5218 -1
False 5118 12853 11752 -1
False 4316 12649 2067 1
False 436 17254 2681 1
False 11728 15164 16764 -1
False 2038 9217 3118 -1
False 14326 15909 12389 -1
True
True
False 11647 17724 8763 -1
True
False 4563 5293 14907 -1
False 7725 7701 4618 -1
True
True
True
True
True
True
F

False 16597 14738 5279 -1
True
False 6501 1032 328 1
False 4215 14883 6898 1
True
True
True
False 15083 14243 10185 -1
False 15421 6110 6641 -1
True
False 19482 2762 571 -1
True
False 514 1743 4393 1
False 4175 5560 2848 1
True
False 6700 9424 18654 -1
False 8001 1855 2554 -1
True
True
False 5266 15007 13065 1
True
False 5334 13535 16583 -1
True
True
True
True
True
False 13343 5445 9992 -1
True
True
True
False 4810 17087 17140 1
False 3990 2485 8120 1
True
True
True
True
False 15948 14841 6749 -1
False 16416 8503 10646 1
True
False 3954 12268 12615 -1
False 9800 18011 18592 -1
True
True
True
True
False 4664 14134 3515 -1
False 6102 7836 4510 -1
False 1753 16297 12308 -1
True
True
True
False 9172 5040 8770 1
False 1202 3355 14029 1
True
False 1916 17020 12786 1
True
True
True
False 9109 16501 5964 -1
False 18910 9455 7966 -1
True
True
True
False 8036 12498 6448 1
True
False 7583 1998 18965 1
True
True
True
True
True
False 15343 7169 15833 1
True
True
False 13749 3080 14477 1
True
False 

In [3]:
tt
tf
ff
ft

2086

In [4]:
tt

3029

In [5]:
tf

1853

In [6]:
ff

3032

In [7]:
ft

2086

In [8]:
(tt+ff)/(ft+tf+tt+ff)

0.6061